In [16]:
import pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [17]:
#Read CSV, index+1 supaya index dimulai dari 1, tidak 0, memudahkan pembacaan dataframe dibawah.
df = pd.read_csv('diabetes.csv')
df.index += 1

In [18]:
#melihat dataframe diabetes.csv
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,diabetes
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1


In [19]:
#target / label. Kolom terakhir/kolom 'diabetes'.
#mengambil dataframe untuk disimpan menjadi array. 
#np_target = df.iloc[:,-1:].values #alternatif
np_target = df['diabetes'].values

In [20]:
#Features. Mengambil kolom 1 - 8 dengan mengakses index. 
np_data = df.iloc[:,:-1].values

In [21]:
#Naive Bayes
clf = GaussianNB()

In [22]:
#Support Vector Classification model
#clf = svm.SVC(kernel='linear', gamma=0.001, C=100)

In [23]:
#Split dataset, 20% testing 80% training.
X_train, X_test, y_train, y_test = train_test_split(np_data,np_target,test_size=0.2, random_state=None)

In [24]:
X_train

array([[0.00e+00, 1.23e+02, 7.20e+01, ..., 3.63e+01, 2.58e-01, 5.20e+01],
       [6.00e+00, 1.17e+02, 9.60e+01, ..., 2.87e+01, 1.57e-01, 3.00e+01],
       [3.00e+00, 1.41e+02, 0.00e+00, ..., 3.00e+01, 7.61e-01, 2.70e+01],
       ...,
       [2.00e+00, 1.08e+02, 6.20e+01, ..., 2.53e+01, 8.81e-01, 2.20e+01],
       [3.00e+00, 1.58e+02, 7.00e+01, ..., 3.55e+01, 3.44e-01, 3.50e+01],
       [1.00e+00, 1.09e+02, 3.80e+01, ..., 2.31e+01, 4.07e-01, 2.60e+01]])

In [25]:
#Cuma buat liat ukuran data train vs data test
len(X_train), len(X_test)

(614, 154)

In [26]:
#Fit model SVM sesuai data training (proses training model)
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [27]:
#CrossValidation model yang sudah dibangun. Dengan parameter scoring=accuracy,precision, dan recall
scoring = ['accuracy', 'precision', 'recall']
scores = cross_validate(clf, X_test, y_test, scoring=scoring, cv=5, return_train_score=True)

In [28]:
#Menampilkan hasil dari cross validation
scores

{'fit_time': array([0.00184917, 0.00075388, 0.00070381, 0.00171876, 0.00071931]),
 'score_time': array([0.00523567, 0.00247812, 0.00480533, 0.00222063, 0.00263906]),
 'test_accuracy': array([0.61290323, 0.83870968, 0.74193548, 0.61290323, 0.7       ]),
 'train_accuracy': array([0.74796748, 0.76422764, 0.7398374 , 0.7398374 , 0.75      ]),
 'test_precision': array([0.42857143, 0.8       , 0.63636364, 0.44444444, 0.6       ]),
 'train_precision': array([0.66666667, 0.74193548, 0.67647059, 0.66666667, 0.6969697 ]),
 'test_recall': array([0.27272727, 0.72727273, 0.63636364, 0.36363636, 0.54545455]),
 'train_recall': array([0.59090909, 0.52272727, 0.52272727, 0.54545455, 0.52272727])}

In [29]:
#menampilkan test_accuracy dari cross validation, memiliki 5 value, tiap value adalah representasi akurasi dari tiap iterasi
scores['test_accuracy']

array([0.61290323, 0.83870968, 0.74193548, 0.61290323, 0.7       ])

In [30]:
#Hasil cross validation berupa dictionary. Kemudian dictionary disimpan menjadi dataframe. Hanya untuk keperluan pembacaan
df_scores = pd.DataFrame.from_dict(scores)
df_scores.index +=1
df_scores

,fit_time,score_time,test_accuracy,train_accuracy,test_precision,train_precision,test_recall,train_recall
1,0.001849,0.005236,0.612903,0.747967,0.428571,0.666667,0.272727,0.590909
2,0.000754,0.002478,0.838710,0.764228,0.800000,0.741935,0.727273,0.522727
3,0.000704,0.004805,0.741935,0.739837,0.636364,0.676471,0.636364,0.522727
4,0.001719,0.002221,0.612903,0.739837,0.444444,0.666667,0.363636,0.545455
5,0.000719,0.002639,0.700000,0.750000,0.600000,0.696970,0.545455,0.522727


In [31]:
#DUMP MODEL YANG SUDAH DIBUAT AGAR MENJADI MODEL PERSISTENCE DAN LANGSUNG DIGUNAKAN DITEMPAT LAIN.
#joblib.dump(clf, 'praktikum1_svc.pkl') 

In [32]:
pickle.dump(clf, open('model_nb.pkl','wb'))

In [34]:
sample = np.array([6,148,72,35,0,34,1,50])
clf.predict_proba(sample.reshape(1,-1))

array([[0.16109847, 0.83890153]])

In [35]:
clf.predict(sample.reshape(1,-1))

array([1])

In [36]:
import pickle
pickle.dump(clf, open('models/model_nb.pickle', 'wb'))